## Welcome to the OpenKIM tutorial at MACH 2023

Let's take a look at what's inside the LAMMPS file. Run the cell below to see:

In [ ]:
!cat in.lammps

Now let's run the file:

In [ ]:
!lmp_mpi < in.lammps

Now let's do a calculation in ASE:

In [ ]:
from ase.calculators.kim import KIM
from ase.lattice.cubic import FaceCenteredCubic
from kim_query import get_lattice_constant_cubic
from ase.optimize import LBFGS
# Define KIM model and get Al fcc lattice parameter for this potential
model = "EAM_Dynamo_ErcolessiAdams_1994_Al__MO_123629422045_005"
calc = KIM(model)
a0 = get_lattice_constant_cubic([model], ["fcc"], ["Al"], ["angstrom"])[0]
# Set up fcc crystal
atoms = FaceCenteredCubic("Al", latticeconstant=a0,size=(9,9,9),pbc=(1,1,0))
atoms.calc = calc
# Compute energy
dyn = LBFGS(atoms)
dyn.run(fmax=1e-8)
print(atoms.get_potential_energy())

## Convex hull

In [ ]:
import numpy as np
from scipy.spatial import ConvexHull
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import matplotlib.pyplot as plt
import json
from curses.ascii import isalpha, isdigit
import requests
from typing import List

#"""

def get_stoich_reduced_list_from_prototype(prototype_label: str) -> List[int]:
    """
    Get numerical list of stoichiometry from prototype label, i.e. "AB3\_...." -> [1,3]

    Args:
        prototype_label:
            AFLOW prototype label

    Returns:
        List of reduced stoichiometric numbers
    """                        
    stoich_reduced_formula = prototype_label.split("_")[0]
    stoich_reduced_list=[]
    stoich_reduced_curr = None
    for char in stoich_reduced_formula:
        if isalpha(char):
            if stoich_reduced_curr is not None:
                if stoich_reduced_curr == 0:
                    stoich_reduced_curr = 1
                stoich_reduced_list.append(stoich_reduced_curr)
            stoich_reduced_curr = 0
        else:
            assert isdigit(char)                            
            stoich_reduced_curr*=10 # will throw an error if we haven't encountered an alphabetical letter, good
            stoich_reduced_curr+=int(char)
    # write final number                    
    if stoich_reduced_curr == 0:
        stoich_reduced_curr = 1
    stoich_reduced_list.append(stoich_reduced_curr)    
    return stoich_reduced_list

response = requests.post("https://query.openkim.org/api",data={'query': '{"property-id":"tag:staff@noreply.openkim.org,2023-02-21:property/cohesive-potential-energy-crystal"}', 'fields': '{"stoichiometric-species.source-value":1,"prototype-label.source-value":1,"formation-potential-energy-per-atom.source-value":1}', 'database': 'data'}).json()
unary_results = {}
compound_results = {}

for result in response:
    species_combo = tuple(result["stoichiometric-species"]["source-value"])
    result_dict = {
                "prototype-label": result["prototype-label"]["source-value"],
                "formation-potential-energy-per-atom": result["formation-potential-energy-per-atom"]["source-value"],
                "stoichiometric-species": result["stoichiometric-species"]["source-value"]
            }
    if len(species_combo)==1:
        if species_combo[0] in unary_results:
            unary_results[species_combo[0]].append(result_dict)
        else:
            unary_results[species_combo[0]] = [result_dict]
    else:
        if species_combo in compound_results:
            compound_results[species_combo].append(result_dict)
        else:
            compound_results[species_combo] = [result_dict]

#loop over all compound species_combos
vertices_dict = {}
for species_combo in compound_results:
    arity = len(species_combo)
    compounds = compound_results[species_combo]
    num_compounds = len(compounds)
    # get reference energies
    elemental_reference_list = []    
    elemental_energies = []
    for element in species_combo:
        elemental_reference = min(unary_results[element],key=lambda tr:tr["formation-potential-energy-per-atom"])
        elemental_reference_list.append(elemental_reference)
        elemental_energies.append(elemental_reference["formation-potential-energy-per-atom"])

    

    stoich_energy_array = np.zeros((num_compounds+2,arity))
    stoich_energy_array[-1,-2]=1

    for (i,compound) in enumerate(compounds):
        stoich_reduced_list = get_stoich_reduced_list_from_prototype(compound["prototype-label"])
        stoich_fractional_list = [elem_num/sum(stoich_reduced_list) for elem_num in stoich_reduced_list]
        reference_energy_per_atom = np.dot(stoich_fractional_list,elemental_energies)
        #set the coordinates to the stoichiometry, skipping first element
        stoich_energy_array[i,:-1]=stoich_fractional_list[1:]
        stoich_energy_array[i,-1]=compound["formation-potential-energy-per-atom"]-reference_energy_per_atom        

    
        
    hull = ConvexHull(stoich_energy_array)
    vertices_dict[species_combo] = hull.vertices.shape[0]
    _ = convex_hull_plot_2d(hull)
    plt.show()
    plt.close("all")